## Create a CNN model and Optimise it using Keras Tuner 

In [12]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 2.2MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=0def27c7b6f338bdb7fb33ae7ab539fb785944657b38e94b227e9206f13da583
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=b094558b52609c966e7adbe739052ddaa08d29bd00810291e2a7e16be789f441
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
print(tf.__version__)

2.2.0


In [0]:
fashion_mnist = keras.datasets.fashion_mnist

In [0]:
(train_images,train_labels),(test_images, test_labels) = fashion_mnist.load_data()

In [0]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [17]:
train_images[0].shape

(28, 28)

In [0]:
train_images = train_images.reshape(len(train_images), 28, 28 , 1)
test_images = test_images.reshape(len(test_images), 28, 28 , 1)

In [0]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [0]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [0]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [22]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.4068 - accuracy: 0.8518 - val_loss: 0.3056 - val_accuracy: 0.8853
Epoch 2/3
1688/1688 [==============================] - 16s 10ms/step - loss: 0.2591 - accuracy: 0.9055 - val_loss: 0.2674 - val_accuracy: 0.9043
Epoch 3/3
1688/1688 [==============================] - 16s 9ms/step - loss: 0.2064 - accuracy: 0.9236 - val_loss: 0.2567 - val_accuracy: 0.9090


Epoch 1/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.4008 - accuracy: 0.8550 - val_loss: 0.2913 - val_accuracy: 0.8927
Epoch 2/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.2570 - accuracy: 0.9060 - val_loss: 0.2607 - val_accuracy: 0.9045
Epoch 3/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1983 - accuracy: 0.9249 - val_loss: 0.2554 - val_accuracy: 0.9057


Epoch 1/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.4889 - accuracy: 0.8270 - val_loss: 0.3615 - val_accuracy: 0.8658
Epoch 2/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.3609 - accuracy: 0.8679 - val_loss: 0.3495 - val_accuracy: 0.8722
Epoch 3/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.3259 - accuracy: 0.8794 - val_loss: 0.3408 - val_accuracy: 0.8765


Epoch 1/3
1688/1688 [==============================] - 14s 9ms/step - loss: 0.3999 - accuracy: 0.8569 - val_loss: 0.2862 - val_accuracy: 0.8917
Epoch 2/3
1688/1688 [==============================] - 14s 8ms/step - loss: 0.2547 - accuracy: 0.9064 - val_loss: 0.2534 - val_accuracy: 0.9065
Epoch 3/3
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1968 - accuracy: 0.9267 - val_loss: 0.2325 - val_accuracy: 0.9183


Epoch 1/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.4025 - accuracy: 0.8551 - val_loss: 0.3113 - val_accuracy: 0.8877
Epoch 2/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.2648 - accuracy: 0.9017 - val_loss: 0.2733 - val_accuracy: 0.9015
Epoch 3/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.2094 - accuracy: 0.9218 - val_loss: 0.2723 - val_accuracy: 0.9060


INFO:tensorflow:Oracle triggered exit


In [0]:
model=tuner_search.get_best_models(num_models=1)[0]

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        57648     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2230368   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 2,289,466
Trainable params: 2,289,466
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.1511 - accuracy: 0.9442 - val_loss: 0.2500 - val_accuracy: 0.9162
Epoch 5/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.1118 - accuracy: 0.9586 - val_loss: 0.3046 - val_accuracy: 0.9092
Epoch 6/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0805 - accuracy: 0.9709 - val_loss: 0.3020 - val_accuracy: 0.9135
Epoch 7/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0610 - accuracy: 0.9777 - val_loss: 0.3312 - val_accuracy: 0.9137
Epoch 8/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0462 - accuracy: 0.9832 - val_loss: 0.3891 - val_accuracy: 0.9137
Epoch 9/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0344 - accuracy: 0.9874 - val_loss: 0.4333 - val_accuracy: 0.9117
Epoch 10/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0280 - accuracy: 0.9902 - val_loss: 0.4753 - v